In [1]:
import numpy as np
import pandas as pd

In [2]:
users = pd.read_csv('../../data/Users.csv', delimiter=';')
books = pd.read_csv('../../data/Books.csv', delimiter=';', dtype={'ISBN': str, 'Title': str, 'Author': str, 'Year': np.int16, 'Publisher': str})
ratings = pd.read_csv('../../data/Ratings.csv', delimiter=';', dtype={'User-ID': np.int32, 'ISBN': str, 'Rating': np.int8})

/var/folders/yn/16crnc792zg2yr78zf0qg0j40000gq/T/ipykernel_42569/2231409931.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('../../data/Users.csv', delimiter=';')


In [3]:
import sys
sys.path.append('../..')

from src.evaluation import average_precision
from sklearn.model_selection import train_test_split

In [4]:
train_ratings, test_ratings = train_test_split(ratings, test_size=0.3, random_state=42)

In [5]:
import src.recommenders.item_collaborative_filtering as icf

model = icf.ItemRecommender()
model.fit(users=users, items=books, ratings=train_ratings)

/Users/oleh.sannikov/Documents/Навчання/Науки про дані/RecSys/CP/rs-ucu-books-recommender/experiments/sunnycows/../../src/recommenders/item_collaborative_filtering.py:29: RuntimeWarning: Mean of empty slice.
  self.means = np.array([normalized_matrix[i].data.mean() for i in range(normalized_matrix.shape[0])])
/Users/oleh.sannikov/miniconda3/envs/recsys/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
test_users = test_ratings['User-ID'].unique()[:100]
len(test_users)

100

In [ ]:
normalized_matrix.shape

(278855, 271379)

In [ ]:
similarity_matrix.shape

(278855, 278855)

In [ ]:
books.shape

(271379, 6)

In [ ]:
best_items = predict(test_users, items=books)

[16264, 37515, 5644, 8858, 4517, 1063, 1448, 16170, 16431, 4798, 16323, 11204, 969, 1353, 129611, 1101, 973, 12896, 7654, 5610, 618, 4590, 7667, 5619, 5624]
[0, 1, 2, 387, 16516, 4, 3, 23437, 3863, 794, 29728, 6314, 6319, 6328, 322, 327, 739, 740, 741, 7406, 1648, 369, 47097]
[0, 1, 2, 3, 4, 1671, 1677, 23, 5798, 5804, 5810, 26163, 3252, 3255, 3256, 3257, 3385, 16056, 12611, 5337, 16605, 15229]
[0, 1, 2, 4, 5, 1924, 6, 8, 1289, 9, 10, 13, 11150, 15, 1294, 1317, 2342, 10549, 54, 12099, 73, 79, 87, 13025, 1402]
[13025, 73, 79, 54, 87]
[14, 50090, 5292, 5302, 41152, 30658, 968, 20424, 21067, 972, 974, 14671, 976, 23383, 42972, 30307, 34406, 1516, 46706, 14969]
[1, 1218, 34, 13025, 1959, 73, 1260, 1230, 1454, 79, 51, 54, 87, 25, 27]
[1, 23074, 3, 4122, 2, 11526, 4, 35304, 12649, 24904, 8, 12, 11, 5, 18130, 4085, 6680, 6522]
[]
[0, 1, 9, 16140, 20, 22, 23, 14492, 11175, 7084, 7090, 13878, 8887, 8383, 22114, 1514, 1519, 2550, 7035, 7038, 1535]
[23808, 21626, 21650, 27157, 23715, 49445, 18855

In [ ]:
len(best_items)
best_items[38781]['ISBN']

73871    0671000403
57042    0373078358
31483    0425176940
2243     0345348109
50150    0441644511
Name: ISBN, dtype: object

In [ ]:
from tqdm import tqdm

rating_mask = test_ratings['Rating'] >= 5

mean_ap = 0
for i in tqdm(range(len(test_users))):
    user_mask = test_ratings['User-ID'] == test_users[i]
    relevant_items = test_ratings.loc[(user_mask & rating_mask)].sort_values(by='Rating', ascending=False)
    if len(relevant_items) == 0:
        continue
    user_id = test_users[i]
    ap = average_precision(best_items[user_id]['ISBN'].to_numpy(), relevant_items['ISBN'].to_numpy())
    mean_ap += ap

mean_ap /= len(test_users)
mean_ap

100%|██████████| 100/100 [00:00<00:00, 366.27it/s]


0.0